# Geospatial data

In [ ]:
from pathlib import Path
from urllib.request import urlretrieve

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import numpy as np
import geopandas as gpd
from shapely.geometry import box

## Load data

In [ ]:
# load data from https://www.naturalearthdata.com/
# url = 'https://naciscdn.org/naturalearth/110m/cultural/ne_110m_admin_0_countries.zip'
# url = 'https://naciscdn.org/naturalearth/50m/cultural/ne_50m_admin_0_countries.zip'
url = 'https://naciscdn.org/naturalearth/10m/cultural/ne_10m_admin_0_countries.zip'

file_name = url.split('/')[-1]
local_path = Path(f'../data/{file_name}')

if not local_path.exists():
    local_path.parent.mkdir(parents=True, exist_ok=True)
    _, _ = urlretrieve(url, filename=str(local_path))

world = gpd.read_file(
    local_path, columns=['ADMIN', 'ISO_A2', 'ISO_A3', 'CONTINENT', 'geometry']
)
world = world.set_geometry('geometry')
world.columns = world.columns.str.lower()

print(f'CRS: {world.crs}')  # EPSG:4326 uses lat/lon coordinates
print(f'Num. rows: {len(world)}')
world.head()

## Plot world

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
world.plot(ax=ax, color='gray')
ax.set(xlabel='Longitude', ylabel='Latitude')
fig.tight_layout()

In [ ]:
world_wgs84 = world.to_crs(epsg=3857)  # Web Mercator projection (meters)
world_wgs84 = world_wgs84[world_wgs84['continent'] != 'Antarctica']  # remove Antarctica for better visualization

world_moll = world.to_crs('ESRI:54009')  # Mollweide projection (meters)

fig, axes = plt.subplots(ncols=2, nrows=1, figsize=(10, 3))
world_wgs84.plot(ax=axes[0], color='gray')
world_moll.plot(ax=axes[1], color='gray')
axes[0].set_title('Web Mercator projection')
axes[1].set_title('Mollweide projection')
fig.tight_layout()

## Plot Europe

In [ ]:
europe = world[(world['continent'].isin(['Europe', 'Asia', 'Africa', 'North America']))]
europe = europe.to_crs(epsg=3035)
europe = europe[europe.geometry.intersects(box(2000000, 1000000, 6500000, 5500000))]
europe = europe.clip(box(2000000, 1000000, 6500000, 5500000))

print(f'CRS: {europe.crs}')
print(f'Num. rows: {len(europe)}')
europe.head()

In [ ]:
colors = plt.cm.gist_earth(np.linspace(0, 0.8, europe['admin'].nunique()))

fig, ax = plt.subplots(figsize=(5, 5))
europe.plot(
    ax=ax, column='admin', cmap=ListedColormap(colors),
    alpha=0.7, edgecolor='white', linewidth=0.4
)
ax.set_xlim(europe.total_bounds[[0, 2]])
ax.set_ylim(europe.total_bounds[[1, 3]])
ax.axis('off')
fig.tight_layout()